In [139]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import re
import sys
import time

In [140]:
session = requests.Session()
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
session.headers.update({'User-Agent': user_agent})

URL = 'https://elezioni.interno.gov.it/regionali/scrutini/20200126/elenchiRI08000000'
page = session.get(URL)

soup = BeautifulSoup(page.content, 'html.parser')

In [141]:
risultatiRegione=[]
a=soup.find(id="sel_province")

provinceHTML=a.findAll('option')

province=pd.DataFrame([(p["value"],p.text) for p in provinceHTML[1:]])
province.columns=["key", "Provincia"]

In [142]:
for (i,p) in province.iterrows():
    print("Provincia di {}".format(p["Provincia"]))
    provinciaURL="https://elezioni.interno.gov.it/regionali/scrutini/20200126/scrutiniRI08"+p["key"]+"0000"
    pageProvincia=session.get(provinciaURL)

    soupProvincia=BeautifulSoup(pageProvincia.content)
    a=soupProvincia.find(id="sel_comuni")
    comuniHTML=a.findAll('option')

    comuni=pd.DataFrame([(p["value"],p.text) for p in comuniHTML[1:]])
    comuni.columns=["key", "Comune"]
    comuni.head()
    for (i,c) in comuni.iterrows():
        print("Comune di {}".format(c["Comune"]))
 
        comuneURL="https://elezioni.interno.gov.it/regionali/scrutini/20200126/scrutiniRI08"+p["key"]+c["key"]
        pageComune=session.get(comuneURL)

        soupComune=BeautifulSoup(pageComune.content)
        scripts=soupComune.findAll("script")
        for s in scripts:
            if "carica_dati" in s.text:
                comuneResURL=re.search(r"(https://.*?)\"",s.text).groups()[0]
                sessionComune=requests.Session()
                user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
                sessionComune.headers.update({'User-Agent': user_agent})
                sessionComune.headers.update({'Referer':comuneURL})
                comuneResHTML=sessionComune.get(comuneResURL)
                comuneResData=json.loads(comuneResHTML.content)
                res={"provinciaKey":p["key"],"provinciaNome":p["Provincia"]}
                res["comuneKey"]=c["key"]
                res["comuneNome"]=c["Comune"]
                res["comuneRes"]=comuneResData
                #print(res)
                risultatiRegione.append(res)
                time.sleep(1)

Provincia di BOLOGNA
Comune di ALTO RENO TERME
Comune di ANZOLA DELL'EMILIA
Comune di ARGELATO
Comune di BARICELLA
Comune di BENTIVOGLIO
Comune di BOLOGNA
Comune di BORGO TOSSIGNANO
Comune di BUDRIO
Comune di CALDERARA DI RENO
Comune di CAMUGNANO
Comune di CASALECCHIO DI RENO
Comune di CASALFIUMANESE
Comune di CASTEL D'AIANO
Comune di CASTEL DEL RIO
Comune di CASTEL DI CASIO
Comune di CASTEL GUELFO DI BOLOGNA
Comune di CASTEL MAGGIORE
Comune di CASTEL SAN PIETRO TERME
Comune di CASTELLO D'ARGILE
Comune di CASTENASO
Comune di CASTIGLIONE DEI PEPOLI
Comune di CREVALCORE
Comune di DOZZA
Comune di FONTANELICE
Comune di GAGGIO MONTANO
Comune di GALLIERA
Comune di GRANAROLO DELL'EMILIA
Comune di GRIZZANA MORANDI
Comune di IMOLA
Comune di LIZZANO IN BELVEDERE
Comune di LOIANO
Comune di MALALBERGO
Comune di MARZABOTTO
Comune di MEDICINA
Comune di MINERBIO
Comune di MOLINELLA
Comune di MONGHIDORO
Comune di MONTE SAN PIETRO
Comune di MONTERENZIO
Comune di MONZUNO
Comune di MORDANO
Comune di OZZA

In [143]:
with open("risultatiRegione.json","w+") as f:
    json.dump(risultatiRegione,f)